<a href="https://colab.research.google.com/github/hadilou/MachineLearning/blob/master/road.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import pandas as pd
from numpy import genfromtxt
import cv2
import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use("gtk")
#import gc
#import seaborn as sns
%matplotlib inline
#%pylab inline
#pal = sns.color_palette()
import skimage.draw
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#from PIL import Image
from tqdm import tqdm

In [3]:
# Root directory of the project
ROOT_DIR = os.path.abspath("C:/Users/Kayode Hadilou ADJE/AnacondaProjects/Mask_RCNN")
print(ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log


C:\Users\Kayode Hadilou ADJE\AnacondaProjects\Mask_RCNN


Using TensorFlow backend.


In [0]:

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [5]:
class RoadConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "road"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 10  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = RoadConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 8
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  128
IMAGE_META_SIZE                23
IMAGE_MIN_DIM                  128
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [128 128   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

In [0]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [0]:

class RoadDataset(utils.Dataset):
    
    def load_road(self, dataset_dir, subset):
        """Load a subset of the Road dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """

        # Add classes. We have only one class to add.
        self.add_class("road",1, "stop")
        self.add_class("road",2,"leftturn")
        self.add_class("road",3,"rightturn")
        self.add_class("road",4,"rail")
        self.add_class("road",5,"35")
        self.add_class("road",6,"forward")
        self.add_class("road",7,"bike")
        self.add_class("road",8,"40")
        self.add_class("road",9,"ped")
        self.add_class("road",10,"xing")
        self.add_class("road",11,"diamond")
        self.add_class("road",12,"car")
        self.add_class("road",13,"pool")
        self.add_class("road",14,"lane")
        self.add_class("road",15,"keep")
        self.add_class("road",16,"clear")
        self.add_class("road",17,"25")
        self.add_class("road",18,"crossing")
        self.add_class("road",19,"X-")
        self.add_class("road",20,"forward&right")
        self.add_class("road",21,"speed")
        self.add_class("road",22,"hump")
        self.add_class("road",23,"30")
        self.add_class("road",24,"slow")
        self.add_class("road",25,"school")
        self.add_class("road",26,"stripe")
        self.add_class("road",27,"forward&left")
        self.add_class("road",28,"yield")
        
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        dataset_dir = dataset_dir + '/'

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        
        
        annotations = pd.read_csv("D:/Datasets/RoadmarkingDataset/New folder/dataset_annotations.txt")
        
        annotations = np.asarray(annotations)
        # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        #annotations = [a for a in annotations if a['label']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            print(a)
            polygons = {
                'all_points_x': [int(a[0]),int(a[2]),int(a[4]),int(a[6])],
                'all_points_y': [int(a[1]),int(a[3]),int(a[5]),int(a[7])],
                'name': 'polygon'
            } 

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            dataDIR = "D:/Datasets/RoadmarkingDataset/RoadMarkingDataset/"
            image_path = dataDIR + a[9] #a[9]=a(file_name)
            print(a[9])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "road",
                image_id=a[9],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)
    def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a road dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "road":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            print(p)
            #rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            #mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "road":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
    


next code builds dataset should be run only once

In [0]:
    def save(filename, output_dir):
        """Resize the image contained in `filename` and save it to the `output_dir`"""
        image = Image.open(os.path.join("D:/Datasets/RoadmarkingDataset/RoadMarkingDataset",filename))
        # Use bilinear interpolation instead of the default "nearest neighbor" method
        image.save(os.path.join(output_dir, filename.split('/')[-1]))
    
    
    
    train_data_dir = os.path.join("D:/Datasets/RoadmarkingDataset/RoadMarkingDataset")
    

    # Get the filenames in each directory (train and test)
    filenames = os.listdir(train_data_dir)
    #print(filenames)
    filenames = [ f for f in filenames if f.endswith('.jpg')]

    # Split the images in 'train_signs' into 60% train and 20% val 20% test
    # Make sure to always shuffle with a fixed seed so that the split is reproducible
    random.seed(230)
    filenames.sort()
    random.shuffle(filenames)

    split = int(0.6 * len(filenames))
    train_filenames = filenames[:split]
    temp = filenames[split:]
    split2 = int(0.2 * len(temp))
    test_filenames = temp[:split2]
    val_filenames = temp[split2:]
     
    filenames = {'train': train_filenames,
                 'val': val_filenames,
                 'test': test_filenames}
    for split in ['train', 'val', 'test']:
        output_dir_split = os.path.join("D:/Datasets/RoadmarkingDataset/RoadMarkingDataset", '{}'.format(split))
        if not os.path.exists(output_dir_split):
            os.mkdir(output_dir_split)
        else:
            print("Warning: dir {} already exists".format(output_dir_split))

        print("Processing {} data, saving preprocessed data to {}".format(split, output_dir_split))
        for filename in tqdm(filenames[split]):
            print(filename)
            save(filename, output_dir_split)

    print("Done building dataset")

Processing train data, saving preprocessed data to D:/Datasets/RoadmarkingDataset/RoadMarkingDataset\train


  0%|                                                                                          | 0/865 [00:00<?, ?it/s]

roadmark_0658.jpg


  0%|                                                                                  | 1/865 [00:00<01:32,  9.35it/s]

roadmark_0107.jpg


...

roadmark_0206.jpg
roadmark_0619.jpg
roadmark_0365.jpg


 99%|███████████████████████████████████████████████████████████████████████████████▎| 114/115 [00:06<00:00, 22.56it/s]

roadmark_0541.jpg


100%|████████████████████████████████████████████████████████████████████████████████| 115/115 [00:06<00:00, 17.08it/s]


Done building dataset


In [9]:
# Training dataset
dataset_train = RoadDataset()
dataset_train.load_road("D:/Datasets/RoadmarkingDataset/RoadMarkingDataset","train")
dataset_train.prepare()

# Validation dataset
dataset_val = RoadDataset()
dataset_val.load_road("D:/Datasets/RoadmarkingDataset/RoadMarkingDataset/test","val")
dataset_val.prepare()

[321.96423300000004 456.40682999999996 433.77786299999997
 335.27536000000003 344.327148 344.327148 317.166656 317.166656 'stop'
 'roadmark_0001.jpg']
roadmark_0001.jpg
[313.97753900000004 453.74459800000005 433.77786299999997 329.950928 350.5
 349.265442 317.166656 317.166656 'stop' 'roadmark_0002.jpg']
roadmark_0002.jpg
[307.32196 449.75125099999997 429.78451500000006 325.957581 352.969147
 351.734558 317.166656 320.870361 'stop' 'roadmark_0003.jpg']
roadmark_0003.jpg
[299.335266 452.413483 431.11563099999995 323.29534900000004 359.141968
 359.141968 320.870361 322.10495 'stop' 'roadmark_0004.jpg']
roadmark_0004.jpg
[295.341919 461.731293 431.11563099999995 316.639771 362.845673 361.611115
 322.10495 325.808655 'stop' 'roadmark_0005.jpg']
roadmark_0005.jpg
[287.355255 455.07571399999995 428.4534 311.315308 367.783966 366.549377
 324.574066 330.746918 'stop' 'roadmark_0006.jpg']
roadmark_0006.jpg
[284.69302400000004 455.07571399999995 429.78451500000006 309.984192
 373.956787 370.2530

In [94]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, class_ids, dataset_train.class_names)

{'all_points_x': [279, 353, 333, 319], 'all_points_y': [310, 338, 320, 341], 'name': 'polygon'}


TypeError: ignored

In [0]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [0]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [0]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\Kayode Hadilou ADJE\AnacondaProjects\Mask_RCNN\logs\road20181126T1434\mask_rcnn_road_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4     

C:\Users\Kayode Hadilou ADJE\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Epoch 1/1


ERROR:root:Error processing image {'id': 'roadmark_1069.jpg', 'source': 'balloon', 'path': 'D:/Datasets/RoadmarkingDataset/RoadMarkingDataset/roadmark_1069.jpg', 'width': 800, 'height': 600, 'polygons': {'all_points_x': [281.964539, 349, 333, 321], 'all_points_y': [300, 343, 321, 338], 'name': 'polygon'}}
Traceback (most recent call last):
  File "C:\Users\Kayode Hadilou ADJE\Anaconda3\lib\site-packages\mask_rcnn-2.1-py3.6.egg\mrcnn\model.py", line 1710, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "C:\Users\Kayode Hadilou ADJE\Anaconda3\lib\site-packages\mask_rcnn-2.1-py3.6.egg\mrcnn\model.py", line 1276, in load_image_gt
    source_class_ids = dataset.source_class_ids[dataset.image_info[image_id]["source"]]
KeyError: 'balloon'
ERROR:root:Error processing image {'id': 'roadmark_0028.jpg', 'source': 'balloon', 'path': 'D:/Datasets/RoadmarkingDataset/RoadMarkingDataset/roadmark_0028.jpg', 'width': 800, 'height': 600, 'polygons': {'all_points_x': [344.59317000000004, 

KeyError: ignored

In [0]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")

In [0]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [0]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [0]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())
AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)

In [0]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [0]:

def getAnnotations():
    dataDIR = "D:/Datasets/RoadmarkingDataset/RoadMarkingDataset"
    annotationsDIR= "D:/Datasets/RoadmarkingDataset/New folder/dataset_annotations.txt"
    annotations = genfromtxt(annotationsDIR, delimiter=',',dtype=str)
    return annotations

    
    
print(getAnnotations())

In [0]:
dataDIR = "D:/Datasets/RoadmarkingDataset/RoadMarkingDataset"
annotationsDIR= "D:/Datasets/RoadmarkingDataset/New folder/dataset_annotations.txt"
df_train = pd.read_csv(annotationsDIR)
#print(df_train.head())
labels = df_train['label'].apply(lambda x: x.split(' '))
labels

In [0]:
from collections import Counter, defaultdict
counts = defaultdict(int)
for l in labels:
    for l2 in l:
        counts[l2] += 1
print(counts)
data=[go.Bar(x=list(counts.keys()), y=list(counts.values()))]
layout=dict(height=800, width=800, title='Distribution of training labels')
fig=dict(data=data, layout=layout)
py.iplot(data, filename='train-label-dist')
py.show()

In [0]:
com = np.zeros([len(counts)]*2)
for i, l in enumerate(list(counts.keys())):
    for i2, l2 in enumerate(list(counts.keys())):
        c = 0
        cy = 0
        for row in labels.values:
            if l in row:
                c += 1
                if l2 in row: cy += 1
        com[i, i2] = cy / c

data=[go.Heatmap(z=com, x=list(counts.keys()), y=list(counts.keys()))]
layout=go.Layout(height=800, width=800, title='Co-occurence matrix of training labels')
fig=dict(data=data, layout=layout)
py.iplot(data, filename='train-com')
py.show()

AttributeError: ignored